In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML
import utilities as ut

Start h2o client

In [3]:
h2o.init(max_mem_size='12G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "17.0.6" 2023-01-17 LTS; Java(TM) SE Runtime Environment (build 17.0.6+9-LTS-190); Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from /home/henrikraa/anaconda3/envs/h2o/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbsb3kvaw
  JVM stdout: /tmp/tmpbsb3kvaw/h2o_henrikraa_started_from_python.out
  JVM stderr: /tmp/tmpbsb3kvaw/h2o_henrikraa_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_henrikraa_psmeq9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


### Load data


In [19]:
y_train = pd.read_csv('y_train.csv')
X_train = pd.read_csv('X_train_selected.csv')
X_test = pd.read_csv('X_test_selected.csv')

y_train = y_train.set_index('date_forecast')
y_train.index = pd.to_datetime(y_train.index)
y_train = y_train.sort_index()

X_train = X_train.set_index('date_forecast')
X_train.index = pd.to_datetime(X_train.index)
X_train = X_train.sort_index()

X_test = X_test.set_index('date_forecast')
X_test.index = pd.to_datetime(X_test.index)
X_test = X_test.sort_index()

In [20]:
X_train_h2o = h2o.H2OFrame(X_train)
X_test_h2o = h2o.H2OFrame(X_test)
y_train_h2o = h2o.H2OFrame(y_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [24]:
# cerate a mergerd dataset usin X_train_h2o and y_train_h2o
df_h2o = X_train_h2o.cbind(y_train_h2o)

In [25]:
df_h2o

temp_solar_interaction,sun_zenith:d,elevation:m,minute,cumulative_precipitation,sun_azimuth:d,sun_equation_of_time,t_1000hPa:K,visibility:m,msl_pressure:hPa,relative_humidity_1000hPa:p,wind_speed_magnitude,day,effective_cloud_cover:p,fresh_snow_24h:cm,wind_effect,fresh_snow_6h:cm,fresh_snow_1h:cm,pv_measurement
0,139.104,24,0,0,12.989,-3.19587,278.5,18220.4,987.3,90.2,2.45967,1,63.6,0,0,0,0,0
0,139.104,7,0,0,12.985,-3.19587,278.4,19275.9,987.4,89.2,2.45967,1,69.6,0,0,0,0,0
0,138.636,24,15,0,18.214,-3.20083,278.4,17083.7,987.1,90,2.7313,1,74.9,0,0,0,0,0
0,138.636,7,15,0,18.206,-3.20083,278.3,17638,987.1,89.1,2.68328,1,78.6,0,0,0,0,0
0,138.026,7,30,0,23.339,-3.20578,278.2,16008.8,986.9,89.1,2.95466,1,86.3,0,0,0,0,0
0,138.026,24,30,0,23.351,-3.20578,278.3,15940.9,986.8,89.9,3.00832,1,84.3,0,0,0,0,0
0,137.281,24,45,0,28.382,-3.21073,278.2,14792,986.6,89.7,3.2311,1,92,0,0,0,0,0
0,137.281,7,45,0,28.367,-3.21073,278.1,14388.5,986.6,89,3.13847,1,92.8,0,0,0,0,0
0,136.409,24,0,0,33.293,-3.21569,278.1,13637.3,986.4,89.6,3.51141,1,98,0,0,0,0,0
0,136.409,7,0,0,33.275,-3.21569,278,12777.3,986.3,88.9,3.36155,1,98.1,0,0,0,0,0


In [26]:
y = "pv_measurement"

x = list(df_h2o.columns)
x.remove(y)

In [28]:
# compare df_h2o.columns with x
print(df_h2o.columns)
print(x)

['temp_solar_interaction', 'sun_zenith:d', 'elevation:m', 'minute', 'cumulative_precipitation', 'sun_azimuth:d', 'sun_equation_of_time', 't_1000hPa:K', 'visibility:m', 'msl_pressure:hPa', 'relative_humidity_1000hPa:p', 'wind_speed_magnitude', 'day', 'effective_cloud_cover:p', 'fresh_snow_24h:cm', 'wind_effect', 'fresh_snow_6h:cm', 'fresh_snow_1h:cm', 'pv_measurement']
['temp_solar_interaction', 'sun_zenith:d', 'elevation:m', 'minute', 'cumulative_precipitation', 'sun_azimuth:d', 'sun_equation_of_time', 't_1000hPa:K', 'visibility:m', 'msl_pressure:hPa', 'relative_humidity_1000hPa:p', 'wind_speed_magnitude', 'day', 'effective_cloud_cover:p', 'fresh_snow_24h:cm', 'wind_effect', 'fresh_snow_6h:cm', 'fresh_snow_1h:cm']


## Run AutoML

In [29]:
aml = H2OAutoML(seed=1, stopping_metric='MAE')
aml.train(x=x, y=y, training_frame=df_h2o)

AutoML progress: |█████ (cancelled)

16:46:54.42: Empty leaderboard.
AutoML was not able to build any model within a max runtime constraint of 3600 seconds, you may want to increase this value before retrying.



H2OJobCancelled: Job<$03017f00000132d4ffffffff$_9180fcb167234e63d48465ee57d14229> was cancelled by the user.

In [18]:
lb = aml.leaderboard
lb.head(rows=15)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_1_20231015_170155,231.768,53716.5,76.3275,nan,53716.5
StackedEnsemble_AllModels_4_AutoML_1_20231015_170155,231.826,53743.3,76.385,nan,53743.3
StackedEnsemble_BestOfFamily_4_AutoML_1_20231015_170155,234.828,55144.2,75.1967,nan,55144.2
StackedEnsemble_BestOfFamily_5_AutoML_1_20231015_170155,236.35,55861.3,75.4507,nan,55861.3
StackedEnsemble_AllModels_2_AutoML_1_20231015_170155,239.951,57576.7,78.6049,nan,57576.7
StackedEnsemble_AllModels_1_AutoML_1_20231015_170155,240.009,57604.3,78.6125,nan,57604.3
GBM_grid_1_AutoML_1_20231015_170155_model_5,240.21,57700.7,76.2509,nan,57700.7
XGBoost_grid_1_AutoML_1_20231015_170155_model_19,240.351,57768.6,79.5444,nan,57768.6
StackedEnsemble_BestOfFamily_3_AutoML_1_20231015_170155,241.122,58139.8,79.0353,nan,58139.8
StackedEnsemble_BestOfFamily_2_AutoML_1_20231015_170155,241.138,58147.4,78.8604,nan,58147.4


In [19]:
aml.leader #Best model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),13/68
# GBM base models (used / total),4/27
# XGBoost base models (used / total),9/33
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/5
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
